In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('indeed_seed.csv')

In [3]:
 df_dd = df.drop_duplicates(subset=['company', 'title'])

In [4]:
desc_series = df_dd['description']
df_dd['js_count'] = desc_series.str.count('[jJ]ava[sS]cript')
df_dd['python_count'] = desc_series.str.count('[Pp]ython')
# convert js_count and python_count columns from float to int
df_dd[['js_count', 'python_count']] = df_dd[['js_count', 'python_count']].fillna(0).astype(int)

In [5]:
def format_hourly(num):
    '''Returns annual salaries converted from hourly rates'''
    return num * 2000 if num > 0 and num < 1000 else num

In [6]:
sal_cap_groups = df_dd['salary'].str.extract(r'\d\d\d?,\d\d\d.-..(\d\d\d?,\d\d\d)|\d\d.-..(\d\d)')
combined = sal_cap_groups[0].combine_first(sal_cap_groups[1])
combined = combined.str.replace(',', '')
formatted_sal = combined.fillna(0).astype(int).map(format_hourly).replace(0, np.nan)
df_dd['formatted_sal'] = formatted_sal
# df_dd['formatted_sal'] = df_dd['formatted_sal'].replace(0, np.nan)
# formatted_sal = combined.fillna(0).astype(int).map(lambda num: num * 2000 if num > 0 and num < 1000 else num).replace(0, np.nan)

In [7]:
grouped = df_dd.groupby(['search_loc', 'search_terms'])
market_sals = grouped['formatted_sal'].mean().sort_values(ascending=False).astype(int)

In [8]:
# market_sals
# df_dd.sort_values(by='js_count', ascending=False)

In [9]:
market_sals.to_csv('market_sals.csv', encoding='utf-8-sig')

In [10]:
df_dd.to_csv('indeed_seed_jup.csv',index=False,encoding='utf-8-sig')